In [1]:
# Keras fix
import tensorflow
from tensorflow.python.ops import control_flow_ops
tensorflow.python.control_flow_ops = control_flow_ops

from keras.models import load_model
from nl.text_gen import generate_from_model
from nl.utils import load_w2v_data, GenerateSamples, print_word_list, parse_text

Using TensorFlow backend.
/home/m20163692/anaconda3/envs/tf/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
print('Loading files...')

text_file = 'text_files/asimov'
model_file = 'models/asimov_model.h5'
mode = 'general_1'

model = load_model(model_file)
seq_len = model.input_shape[1]
input_dim = model.input_shape[2]

word2idx, idx2word, embeddings = load_w2v_data('w2v_embs/asimov/asimov_vocab.json',
                                               'w2v_embs/asimov/asimov_embeddings.npz')

Loading files...


In [10]:
# Random samples from text_file
n_samples = 3
n_words = 100
print_truth = True
n_truth_words = 30
temperatures = [0.25, 0.5, 0.75]

samples = GenerateSamples(word2idx, text_file, mode, 'dir', n_samples, n_words)

for idx, sample in enumerate(samples):
    print('-' * 80)
    print('SAMPLE %d:' % (idx + 1))
    print_word_list(sample[:seq_len])
    
    if print_truth:
        print('\nTRUTH:')
        print_word_list(sample[:seq_len + n_truth_words])
    
    for temperature in temperatures:
        print('\nGENERATED FROM MODEL (temperature = %f):' % temperature)
        model_sample = generate_from_model(sample, model, embeddings, idx2word,
                                           word2idx, n_words, temperature, mode=mode)
        print_word_list(model_sample)
    
    print('-' * 80)

--------------------------------------------------------------------------------
SAMPLE 1:
— nadie viéndola lo adivinaría. debe de haber sido muy hermosa

TRUTH:
— nadie viéndola lo adivinaría. debe de haber sido muy hermosa en su juventud. 
 — nunca me lo dijeron, d. g. graciosa, mona, siempre creí que era a lo más que podía aspirar...

GENERATED FROM MODEL (temperature = 0.250000):
— nadie viéndola lo adivinaría. debe de haber sido muy hermosa. 
 — ¿y qué? 
 — no, no lo sé. 
 — ¿y qué? 
 — ¿qué? — preguntó seldon. 
 — ¿qué? — preguntó seldon. 
 — ¿qué? 
 — no, no lo sé. 
 — ¿qué? 
 — ¿y qué? 
 — ¿qué? ¿qué? 
 — ¿qué? 
 — no se lo. 
 — ¿qué? — preguntó seldon. 
 — ¿qué? — preguntó seldon.

GENERATED FROM MODEL (temperature = 0.500000):
— nadie viéndola lo adivinaría. debe de haber sido muy hermosa en los mundos del espacio. 
 — ¿qué? — preguntó pelorat —. ¿qué, lo que no puedo hacer? ¿qué? 
 — no. 
 — ¿y no es eso lo que me lo ha dicho? 
 — no, no lo sé. 
 — ¿qué? 
 — no lo sé. ¿qué e

In [11]:
# Using custom text
text = 'dentro de un poco mas de un día'
n_words = 100
temperatures = [0.25, 0.5, 0.75]

print('SOURCE TEXT:\n%s' % text)

for temperature in temperatures:
    print('\nGENERATED FROM MODEL (temperature = %f):' % temperature)
    model_sample = generate_from_model(text, model, embeddings, idx2word, word2idx,
                                       n_words, temperature, truncating='post',
                                       custom_text=True, mode='general_2')
    print_word_list(model_sample)

SOURCE TEXT:
dentro de un poco mas de un día

GENERATED FROM MODEL (temperature = 0.250000):
dentro de un poco mas de un día, un mundo de la tierra, que lo que se había a la tierra no era más que una vida. 
 — ¿y qué? 
 — ¿qué? 
 — no lo sé. 
 — ¿qué? — preguntó seldon. 
 — ¿qué? — preguntó seldon. 
 — ¿qué? 
 — no lo sé. 
 — ¿qué? 
 — ¿y qué? 
 — ¿qué? — preguntó seldon. 
 — no. 
 — ¿y qué? 
 —

GENERATED FROM MODEL (temperature = 0.500000):
dentro de un poco mas de un día, la que se había a su la tierra, y, por lo tanto, el doctor han fastolfe, no en la tierra. 
 — ¿qué? 
 — es un hombre, una vez, de una gran idea, la más importante. 
 — ¿y por qué no? 
 — el doctor fastolfe está en el hombre en que me parece que no es el único que es la primera fundación. 
 — ¿qué? — preguntó seldon. 
 — no. 
 — ¿por qué?

GENERATED FROM MODEL (temperature = 0.750000):
dentro de un poco mas de un día, aunque no vida, él no le había que la es más importante. 
 — ¿qué? — preguntó estaba de pie y trevi